In [7]:
#Miami, December 27th, 2019
#Renato Kogeyama

# Due bad performance, I am updating the script to more efficient coding

# This script generates a citation ouput with patent|backward citation|cumulated backward citation | year
# The output generates a citation record by year, considering direct and indirect citation


In [8]:
import pandas as pd
import re
import matplotlib.pyplot as plot

import sys
sys.path.append('/home/rkogeyam/scripts/')
from sampler import sampler


In [9]:
%matplotlib inline

In [10]:
fname = r'/home/rkogeyam/PATENT_CITATION/data/uspatentcitation.tsv'
dst = '/home/rkogeyam/PATENT_CITATION/cit_tree.csv'

In [ ]:
%%time
# df=pd.read_csv(fname, sep='\t')

sample_size=100
df=sampler(fname, sample_size, sep='\t')

In [ ]:
df['patent_id']=df['patent_id'].astype(str)

In [ ]:
df['citation_id']=df['citation_id'].astype(str)

In [ ]:
df.shape

In [ ]:
df.dtypes

In [10]:
# drop rows for which column citation_id has a length smaller than 5
indexNames = df[df['citation_id'].map(len) < 5].index
df.drop(indexNames , inplace=True)

In [11]:
# drop rows for which column patent_id has a length smaller than 5
indexNames = df[df['patent_id'].map(len) < 5].index
df.drop(indexNames , inplace=True)

In [12]:
df.shape

(91445558, 9)

In [13]:
%%time
#stripping non desired characters but keeping the originals for later check
#could merge unwanted information
cleaning_patent=lambda x:re.sub('([^a-zA-Z0-9]+)', "", x)
df['citation_id']=df['citation_id'].apply(cleaning_patent)
df['patent_id']=df['patent_id'].apply(cleaning_patent)
# df['citation_id'] = df['citation_id'].str.extract('([a-zA-Z0-9]+)', expand=True)
# df['patent_id'] = df['patent_id'].str.extract('([a-zA-Z0-9]+)', expand=True)

CPU times: user 7min 58s, sys: 118 ms, total: 7min 58s
Wall time: 7min 58s


In [14]:
#stripping trailing white spaces
df['patent_id'] = df['patent_id'].str.strip()
df['citation_id'] = df['citation_id'].str.strip()

In [15]:
%%time
#to avoid problems when converting object to datetime format
#replacing day or month would not affect the output
df.date.replace({'-00':'-01'}, regex=True, inplace=True)
#replacing the year could be more problematic
#for now, I am just dropping


CPU times: user 1min 7s, sys: 8 ms, total: 1min 7s
Wall time: 1min 7s


In [16]:
%%time
df['date']=pd.to_datetime(df.date,format='%Y-%m-%d', errors='coerce') #attention to date format - original data is year-month-day
# df['date_format']=pd.to_datetime(df.date,format='%Y-%m-%d', errors='coerce')

CPU times: user 31.8 s, sys: 1.05 s, total: 32.9 s
Wall time: 32.8 s


In [17]:
df.shape

(91445558, 9)

In [18]:
%%time
#first, i'll run only patents with some kind of citation
#as we have many zero citations in the dataset, this should decrease processing time

back_citation=df.groupby(['citation_id']).count().iloc[:, 1].reset_index() #reset_index returns dataframe
forw_citation=df.groupby(['patent_id']).count().iloc[:, 1].reset_index()
# back_citation=df.groupby(['citation_id']).count() 

CPU times: user 3min 13s, sys: 4.18 s, total: 3min 17s
Wall time: 3min 17s


In [19]:
back_citation.rename(columns={'patent_id': 'back_citation'}, inplace=True) #index: 'citation_id'
forw_citation.rename(columns={'citation_id': 'forw_citation'}, inplace=True) #index: 'patent_id'

In [ ]:
back_citation.head()

In [ ]:
forw_citation.head()

In [20]:
%%time
#this is the trick calculation
#I match the total of citations received by the citing patent (identified by patent_id)
#In the next step I sum all citations received by the citing patents

df=df.merge(back_citation, how='inner', left_on='patent_id', right_on='citation_id')
df.rename(columns={'back_citation': 'parent_back_citation'}, inplace=True)

CPU times: user 4min 4s, sys: 4.99 s, total: 4min 9s
Wall time: 4min 9s


In [ ]:
outcome=df.groupby('citation_id').sum()

In [ ]:
outcome.head()

In [26]:
# lastly, i merge the number of citations to patent_level_df
# patent_level has the information about the citing patent

outcome=back_citation.merge(df, how='outer', left_on='citation_id', right_index=True)
outcome=back_citation.merge(forw_citation, how='outer', left_on='citation_id', right_on='patent_id')

In [27]:
# outcome.fillna(0, inplace=True)

In [28]:
outcome.head()

,citation_id,back_citation,patent_id,forw_citation
0,0000000,1.0,NaN,NaN
1,000004,1.0,NaN,NaN
2,000006,1.0,NaN,NaN
3,0000482,1.0,NaN,NaN
4,0000H94,1.0,NaN,NaN


In [29]:
outcome.drop(['patent_id'], axis=1, inplace=True)

In [30]:
# outcome=outcome.loc[:,['citation_id','back_citation','citation_id_y','parent_back_citation']].set_index('citation_id')

In [31]:
outcome.describe()

/home/applications/anaconda3/4.2.0/lib/python3.5/site-packages/numpy/lib/function_base.py:4269: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


,back_citation,forw_citation
count,7.049261e+06,6.188700e+06
mean,1.297236e+01,1.477621e+01
std,3.053004e+01,4.195074e+01
min,1.000000e+00,1.000000e+00
25%,NaN,NaN
50%,NaN,NaN
75%,NaN,NaN
max,3.922000e+03,5.811000e+03


In [32]:
outcome.head()

,citation_id,back_citation,forw_citation
0,0000000,1.0,NaN
1,000004,1.0,NaN
2,000006,1.0,NaN
3,0000482,1.0,NaN
4,0000H94,1.0,NaN


In [33]:
%%time
outcome.to_csv(dst)

CPU times: user 42 s, sys: 79 ms, total: 42 s
Wall time: 42.2 s
